In [1]:
!pip install pyspark
!pip install pyarrow
!pip install -q findspark

     |████████████████████████████████| 212.3MB 76kB/s 
     |████████████████████████████████| 204kB 19.6MB/s 
  Created wheel for pyspark: filename=pyspark-3.1.1-py2.py3-none-any.whl size=212767604 sha256=abb1856bf529e060c5fe9037ab33d4605f44fd79f6927ac56ab515d51f6e4003
  Stored in directory: /root/.cache/pip/wheels/0b/90/c0/01de724414ef122bd05f056541fb6a0ecf47c7ca655f8b3c0f
Successfully built pyspark


In [2]:
from pyspark.sql import SparkSession

In [3]:
spark=SparkSession.builder.appName("Missing Data").getOrCreate()

In [4]:
df=spark.read.csv("/content/ContainsNull.csv",inferSchema=True,header=True)

In [5]:
df.show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp1| John| null|
|emp2| null| null|
|emp3| null|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+



In [7]:
df.na.drop().show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp4|Cindy|456.0|
+----+-----+-----+



In [8]:
df.na.drop(thresh=2).show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp1| John| null|
|emp3| null|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+



In [9]:
df.na.drop(how="any").show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp4|Cindy|456.0|
+----+-----+-----+



In [10]:
df.na.drop(how="all").show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp1| John| null|
|emp2| null| null|
|emp3| null|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+



In [11]:
df.na.drop(subset=['Sales']).show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp3| null|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+



In [12]:
df.printSchema()

root
 |-- Id: string (nullable = true)
 |-- Name: string (nullable = true)
 |-- Sales: double (nullable = true)



In [13]:
df.na.fill("FILLLLL").show()

+----+-------+-----+
|  Id|   Name|Sales|
+----+-------+-----+
|emp1|   John| null|
|emp2|FILLLLL| null|
|emp3|FILLLLL|345.0|
|emp4|  Cindy|456.0|
+----+-------+-----+



In [14]:
df.na.fill(10000).show()

+----+-----+-------+
|  Id| Name|  Sales|
+----+-----+-------+
|emp1| John|10000.0|
|emp2| null|10000.0|
|emp3| null|  345.0|
|emp4|Cindy|  456.0|
+----+-----+-------+



In [15]:
df.na.fill("NO NAME",subset=["Name"]).show()

+----+-------+-----+
|  Id|   Name|Sales|
+----+-------+-----+
|emp1|   John| null|
|emp2|NO NAME| null|
|emp3|NO NAME|345.0|
|emp4|  Cindy|456.0|
+----+-------+-----+



In [16]:
from pyspark.sql.functions import mean

In [18]:
mean_value=df.select(mean(df['Sales'])).collect()

In [20]:
mean_value=mean_value[0][0]

In [22]:
df.na.fill(mean_value,["Sales"]).show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp1| John|400.5|
|emp2| null|400.5|
|emp3| null|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+

